# 네이버 쇼핑리뷰 감성분석
- GRU
- Mecab 형태소분석(리눅스에서만 구동)

In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 17.82 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
bash: install_mecab-ko_on_colab_light_220111.sh: No such file or directory


In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

In [17]:
import re
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [4]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'

In [18]:
df = pd.read_table(url, names=['rating', 'reviews'])
df.head(3)

,rating,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...


In [19]:
# 평점이 4점 이상인 데이터 = 1
df['label'] = df.rating.apply(lambda x: 1 if x >= 4 else 0)
df.head()

,rating,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [27]:
df.label.value_counts()

0    99955
1    99953
Name: label, dtype: int64

## 1.데이터 전처리

In [20]:
df.isna().sum()

rating     0
reviews    0
label      0
dtype: int64

In [21]:
df.reviews.nunique(),len(df)

(199908, 200000)

In [22]:
# 중복 제거
df = df.drop_duplicates(subset=['reviews'])

In [23]:
df.reviews = df.reviews.str.replace('[^ㄱ-ㅎ가-힣 ]', ' ')

In [24]:
df.head()

,rating,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서 개 더 구매했어요 이가격에 대박입니다 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전...,0
4,5,민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [25]:
df.isna().sum().sum() # replace이후 na값이 생겼는지 확인용

0

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.reviews.values, df.label.values, stratify = df.label.values ,
    test_size = 0.2, random_state = 2022
)

In [31]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((159926,), (39982,), (159926,), (39982,))

## 2.Tokenizer

In [32]:
from konlpy.tag import Mecab
mecab = Mecab()

In [33]:
stopwords = ['이','가','의','은','들','는','좀','잘','걍', '과','도','를','으로','자','에','와','한','하다','을','ㅋ']

In [35]:
from tqdm import tqdm

train_data = []
for sentence in tqdm(X_train):
    morphs = mecab.morphs(sentence)
    tmp_X =  [word for word in morphs if word not in stopwords]
    train_data.append(tmp_X)

100%|██████████| 159926/159926 [00:24<00:00, 6600.06it/s]


In [36]:
test_data = []
for sentence in tqdm(X_test):
    morphs = mecab.morphs(sentence)
    tmp_X =  [word for word in morphs if word not in stopwords]
    test_data.append(tmp_X)

100%|██████████| 39982/39982 [00:06<00:00, 6607.19it/s]


In [37]:
train_data[0]

['재', '구매', '늘', '먹', '던', '거', '예요', '밥맛', '좋', '아요']

- Keras - Tokenizer

In [38]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer
token = Tokenizer()
token.fit_on_texts(train_data)

In [47]:
len(token.word_index)

38133

In [48]:
# 빈도수가 3미만인 단어 개수 파악
threshold = 3
total_cnt = len(token.word_index)   # 294186
rare_cnt = 0        # 등장빈도가 thredhold보다 적은 단어의 수
total_freq = 0      # 훈련 데이터의 전체 단어의 빈도수의 합
rare_freq = 0       # 등장 빈도가 threshold보다 작은 단어의 빈도 수의 합 

In [49]:
for key, value in token.word_counts.items():
    total_freq += value
    if value <= 3:
        rare_cnt += 1
        rare_freq += value

In [50]:
print('단어 집합(vocabulary)의 크기 :', total_cnt)
print(f'등장 빈도가 {threshold - 1}번 이하인 희귀 단어의 수: {rare_cnt}')
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 38133
등장 빈도가 2번 이하인 희귀 단어의 수: 23439
단어 집합에서 희귀 단어의 비율: 61.466446385020845
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.2828940571639347


- 등장빈도가 2회 이하인 단어는 제외하고 토큰화

In [51]:
# 0번은 패딩 토큰, 
# 1번 OOV(out of value) 토큰 고려
vocab_size = total_cnt - rare_cnt + 2
vocab_size

14696

In [52]:
token = Tokenizer(num_words=vocab_size, oov_token='OOV')
token.fit_on_texts(train_data)
X_train = token.texts_to_sequences(train_data)
X_test = token.texts_to_sequences(test_data)

## 3.Padding

In [53]:
max(len(s) for s in X_train) , sum(map(len, X_train)) / len(X_train)

(84, 16.34172054575241)

In [54]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 50
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((159926, 50), (39982, 50))

## 4.모델설정 - GRU

In [55]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

In [57]:
model = Sequential([
    Embedding(vocab_size, 100, input_length=max_len),
    GRU(128),
    Dense(1, activation = 'sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 100)           1469600   
                                                                 
 gru_1 (GRU)                 (None, 128)               88320     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,558,049
Trainable params: 1,558,049
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])
model_path = 'best-naver-shop.h5'
mc = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
es = EarlyStopping(patience=3)

hist = model.fit(X_train, y_train, validation_split = 0.2,
                 epochs = 30, batch_size =128, callbacks = [mc, es])

Epoch 1/30
1000/1000 [==============================] - ETA: 0s - loss: 0.2847 - accuracy: 0.8896
Epoch 1: val_loss improved from inf to 0.23782, saving model to best-naver-shop.h5
1000/1000 [==============================] - 189s 186ms/step - loss: 0.2847 - accuracy: 0.8896 - val_loss: 0.2378 - val_accuracy: 0.9125
Epoch 2/30
1000/1000 [==============================] - ETA: 0s - loss: 0.2106 - accuracy: 0.9251
Epoch 2: val_loss improved from 0.23782 to 0.22730, saving model to best-naver-shop.h5
1000/1000 [==============================] - 186s 186ms/step - loss: 0.2106 - accuracy: 0.9251 - val_loss: 0.2273 - val_accuracy: 0.9161
Epoch 3/30
1000/1000 [==============================] - ETA: 0s - loss: 0.1796 - accuracy: 0.9371
Epoch 3: val_loss did not improve from 0.22730
1000/1000 [==============================] - 185s 185ms/step - loss: 0.1796 - accuracy: 0.9371 - val_loss: 0.2415 - val_accuracy: 0.9123
Epoch 4/30
1000/1000 [==============================] - ETA: 0s - loss: 0.1563

In [61]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

1250/1250 [==============================] - 21s 16ms/step - loss: 0.2336 - accuracy: 0.9136


[0.23358172178268433, 0.9136361479759216]

In [66]:
def sentiment_predict(text, tokenizer = token, max_len = max_len):
    text = re.sub('[^ㄱ-ㅎ가-힣]', ' ', text).strip()
    morphs = mecab.morphs(text)
    text = [w for w in morphs if w not in stopwords]
    encoded = token.texts_to_sequences([text])
    padded = pad_sequences(encoded, maxlen = max_len)
    score = best_model.predict(padded)[0,0]
    return '긍정' if score > 0.5 else '부정'

In [67]:
print(sentiment_predict('진짜 배송도 늦고 개짜증나네요. 뭐 이런 걸 상품이라고 만듬?'))
print(sentiment_predict('ㅁㄴㅇㄻㄴㅇㄻㄴㅇ리뷰쓰기도 귀찮아'))
print(sentiment_predict('판매자님... 너무 짱이에요.. 대박나삼'))

부정
부정
긍정
